In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.options.display.expand_frame_repr = False

In [3]:
import my_lib.lib_sql as ls

In [10]:
model = '11-5225'
model_number = '2016'

cols = ls.get_column_headers(ls.engine)
cols.remove('id')
#cols.remove('gr_vagon')
cols

['model',
 'model_number',
 'name',
 'gr_vagon',
 'Тип вагона',
 'Дополнительная характеристика',
 'Особенность модели',
 'Учётная специализация модели',
 'Завод-изготовитель',
 'Материал кузова',
 'Тележка',
 'Осность вагона',
 'Ширина колеи',
 'Наличие переходной площадки',
 'Наличие стояночного тормоза',
 'Конструкционная скорость',
 'Тара вагона (минимальная)',
 'Тара вагона (максимальная)',
 'Грузоподъёмность',
 'Объём',
 'Площадь пола',
 'Высота',
 'Ширина',
 'Длина',
 'Габарит по ГОСТ 9238-2013',
 'Длина по осям автосцепок',
 'Размер дверного проёма',
 'Год начала серийного производства',
 'Нормативный срок службы',
 'url',
 'Номер проекта',
 'Технические условия',
 'Возможность установки буферов',
 'Максимальная расчетная статическая нагрузка от колесной пары на рельсы',
 'Максимальная расчетная погонная нагрузка',
 'База вагона',
 'Грузоподъёмность секции',
 'База секции',
 'Объём одной секции',
 'Площадь пола секции вагона',
 'Габарит по ГОСТ 9238-2013 при закрытой крыше',
 '

In [25]:
wagon = dict()
with ls.Session(ls.engine) as session:
    q = session.query(ls.Vagon).all()
    for qi in q:
        attr = getattr(qi, 'Завод-изготовитель')
        try:
            wagon[attr]=attr.count('клейм')
            if  wagon[attr]>1:
                print(attr, wagon[attr])
        except:
            print('model', getattr(qi, 'model'), 'model_number', getattr(qi, 'model_number'))
wagon

model 15-1443-6У model_number 9017.0
model 15-776-9У model_number 9054.0
model 15-777 model_number 444.0
model 15-1443-9У model_number 9011.0
model 15-1428-9У model_number 9009.0
model 15-1443-9У model_number 9013.0
model 15-1443-8У model_number 9026.0
model 15-1443-9У model_number 9012.0
model 15-1443-9У model_number 9010.0
model 14-9068 model_number 9595.0
model 31-673-18 model_number 9590.0
model 31-673-18 model_number 9591.0
model 31-674-18 model_number 9592.0
model 31-674-18 model_number 9593.0
model 31-656-18 model_number 9588.0
model 31-656-18 model_number 9587.0
model 11-Н003 model_number 59.0
model 31-638-18 model_number 9583.0
model 31-638-18 model_number 9584.0
model 19-756 model_number None
model 17-4025 model_number None


{'Акционерное общество "Научно-производственная корпорация "Уралвагонзавод" имени Ф.Э. Дзержинского (клеймо 5)': 1,
 'Акционерное общество Алтайского вагоностроения (клеймо 22)': 1,
 'Открытое акционерное общество "Рузаевский завод химического машиностроения" (клеймо 1167)': 1,
 'Общество с ограниченной ответственностью «Опытно-механический завод «Карпаты» (Украина) (клеймо 1383)': 1,
 'Общество ограниченной ответственности "Вагоностроительная компания" г. Рустави (клеймо 1282)': 1,
 'Общество с ограниченной ответственностью "АЗОВВАГОН" (клеймо 2161)': 1,
 'Публичное акционерное общество "Крюковский вагоностроительный завод" (клеймо 27)': 1,
 'Частное акционерное общество "Днепровагонмаш" (клеймо 133)': 1,
 'Армавирский машиностроительный завод" филиала Открытого акционерного общество "Новозыбковский машиностроительный завод (клеймо 1535)': 1,
 'СЗАО "Могилёвский вагоностроительный завод" (клеймо 1284)': 1,
 'Акционерное общество «Тихвинский вагоностроительный завод» (клеймо 1378)': 1,

In [26]:
url = f'https://vagon.by/factories/'
page = requests.get(url)
print(page.status_code)
soup = BeautifulSoup(page.text, "html.parser")
t = soup.find('table')

200


In [27]:
len(t.findAll('a'))

2688

In [35]:
pd.read_html(url)[0].to_csv('excel_files/factories.csv', index=False)

In [25]:
cat = ['Общие сведения о предприятии', 'Вагоны, построенные (модернизированные) на предприятии', 'Виды выполняемых работ (грузовые вагоны)']

In [37]:
rows = []

for i in t.findAll('a')[:5]:
    url_factory = 'https://vagon.by' + i.get('href')

    page_factory = requests.get(url_factory)
    soup_factory = BeautifulSoup(page_factory.text, "html.parser")
    article_factory = soup_factory.find('article')
    factory_info = dict()
    
    factory_info['Название пред­прия­тия'] = article_factory.find('h1', {'itemprop': 'legalName'}).text

    for c in article_factory.findAll('h3'):
        if c.text not in cat:
            print(f'Новая категория: {c}')

    h3 = article_factory
    print(h3)
    print()





<article itemscope="" itemtype="http://schema.org/Organization"><h1 itemprop="legalName">Днепропетровский металлургический завод</h1><h3>Общие сведения о предприятии</h3>
Условный код предприятия (клеймо): 3<br/>
Сокращённое название: <span itemprop="name">Днепропетровский МРГЗ</span><br/>Официальный веб-сайт: <a href="http://dmz-petrovka.dp.ua" itemprop="sameAs" rel="nofollow" target="_blank">http://dmz-petrovka.dp.ua</a><br/>Станция примыкания: <a href="https://railwayz.info/photolines/station/8118" target="_blank">станция Каменское</a>, <a href="https://railwayz.info/photolines/rw/45" target="_blank">Приднепровская ж.д.</a><h3>Вагоны, построенные (модернизированные) на предприятии</h3><br/><a href="/model/14-9062">Транспортёр для контейнера HI-TRAC 190 мод. 14-9062</a><br/></article>

<article itemscope="" itemtype="http://schema.org/Organization"><h1 itemprop="legalName">Акционерное общество "Вологодский вагоноремонтный завод"</h1><h3>Общие сведения о предприятии</h3>
Условный код 

In [46]:
a = '</article>'
len(a)

10

In [55]:
url_factory = 'https://vagon.by/factories/view/5'

page_factory = requests.get(url_factory)
soup_factory = BeautifulSoup(page_factory.text, "html.parser")
article_factory = soup_factory.find('article')
factory_info = dict()

factory_info['Название пред­прия­тия'] = article_factory.find('h1', {'itemprop': 'legalName'}).text

for c in article_factory.findAll('h3'):
    if c.text not in cat:
        print(f'Новая категория: {c}')

""" h3 = str(article_factory)[64:-10]
for i in h3.split('><'):
    print(i)
 """

Новая категория: <h1 itemprop="legalName">Акционерное общество "Научно-производственная корпорация "Уралвагонзавод" имени Ф.Э. Дзержинского</h1>


" h3 = str(article_factory)[64:-10]\nfor i in h3.split('><'):\n    print(i)\n "

In [60]:
html = article_factory
for tag in html.find_all(True):
    content = tag.get_text(strip=True)
    print('content', content)
    print()

content Акционерное общество "Научно-производственная корпорация "Уралвагонзавод" имени Ф.Э. Дзержинского

content Общие сведения о предприятии

content 

content АО "НПК"Уралвагонзавод""

content 

content http://www.uvz.ru

content 

content станция Вагонозавод

content Свердловская ж.д.

content Виды выполняемых работ (грузовые вагоны)

content Капитальный ремонт (КР)Постройка вагоновУстановка датчиков САИПСИзготовление черновой осиИзготовление боковой рамыИзготовление надрессорной балкиФормирование и ремонт колёсных пар со сменой элементовПолное освидетельствование колёсных парИзготовление тормозной продукции

content Капитальный ремонт (КР)

content Постройка вагонов

content Установка датчиков САИПС

content Изготовление черновой оси

content Изготовление боковой рамы

content Изготовление надрессорной балки

content Формирование и ремонт колёсных пар со сменой элементов

content Полное освидетельствование колёсных пар

content Изготовление тормозной продукции

content Вагоны, по

In [58]:
article_factory.find('text()')

In [50]:
str(article_factory)

'<article itemscope="" itemtype="http://schema.org/Organization"><h1 itemprop="legalName">Акционерное общество "Научно-производственная корпорация "Уралвагонзавод" имени Ф.Э. Дзержинского</h1><h3>Общие сведения о предприятии</h3>\nУсловный код предприятия (клеймо): 5<br/>\nСокращённое название: <span itemprop="name">АО "НПК"Уралвагонзавод""</span><br/>Официальный веб-сайт: <a href="http://www.uvz.ru" itemprop="sameAs" rel="nofollow" target="_blank">http://www.uvz.ru</a><br/>Станция примыкания: <a href="https://railwayz.info/photolines/station/12427" target="_blank">станция\xa0Вагонозавод</a>, <a href="https://railwayz.info/photolines/rw/76" target="_blank">Свердловская ж.д.</a><h3>Виды выполняемых работ (грузовые вагоны)</h3><ul><li>Капитальный ремонт (КР)</li><li>Постройка вагонов</li><li>Установка датчиков САИПС</li><li>Изготовление черновой оси</li><li>Изготовление боковой рамы</li><li>Изготовление надрессорной балки</li><li>Формирование и ремонт колёсных пар со сменой элементов</li

In [53]:
with open('readme.txt', 'w') as f:
    f.write(str(article_factory))